# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

Term frequency: $tf_{t,d} = log_{10} count(t,d)+1$  
Inverse document frequency: $idf_{t} = log_{10} (N/df_{t})$  
TF-idfweighted value: $w_{t,d} = tf_{t,d} x idf_{t}$   

N - 文档总数  
df_t - 出现t这个词的文档数目

#### 2. LDA算法的基本假设是什么？

- 文档-主题分布：每个文档的主题服从一个概率分布  
- 主题-词分布： 每个主题的词服从一个概率分布

#### 3. 在TextRank算法中构建图的权重是如何得到的？

$PR(P_{i},t+1) = (1-d)+d*\sum_{j\in In(P_{i})}\frac{1}{|Out(P_j)|}PR(P_j,t)$  

$P_{j}$ - 与$P_{i}$相连的点   
$Out(P_{j})$ - $P_{j}$指向别的点的线数目

#### 4. 什么是命名实体识别？ 有什么应用场景？

命名实体识别（Named Entity Recognition，简称NER），又称作“专名识别”，是指识别文本中具有特定意义的实体，主要包括人名、地名、机构名、专有名词等。简单的讲，就是识别自然文本中的实体指称的边界和类别。  
应用：  
- 人名、地名、机构名、专有名词
- 生物领域：基因名字、蛋白质名字
- 教育领域： 大学课程的代码

#### 5.NLP主要有哪几类任务 ？

- 分类任务：文本分类、情感分析
- 生成任务：问答系统

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

https://blog.csdn.net/qq_36910634/article/details/97764251  
https://blog.csdn.net/y12345678904/article/details/77855936

In [5]:
import numpy as np
import pandas as pd
import re
import random
import jieba
import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec

**Step 1. Preprocessing**

In [26]:
#randomly select 100 news
filename = ".\sqlResult_1558435.csv"
content = pd.read_csv(filename,encoding='gb18030')
content_100 = content.sample(n=100,replace=False)
#content_100.to_csv('./content_100.csv',index=0)

#clean and cut text
filename = ".\content_100.csv"
content = pd.read_csv(filename,encoding='utf-8')
content = content['content'].tolist()

#get stopwords list
stop_words = []
with open('./baidu_stopwords.txt',encoding='utf-8') as f:
    for word in f.readlines():
        stop_words.append(word.strip())

def token(string):
    token = ''.join(re.findall(r'[\d|\w]+',str(string)))  #不加str运行content = [token(n) for n in content]报错
    return token

def cut(string):
    return " ".join([w for w in jieba.cut(string) if w not in stop_words]) 

content = [token(n) for n in content]
content = [cut(n) for n in content]

with open('.\content_100_cut.txt','w',encoding="utf-8") as f:
    for a in content:
        f.write(a)

# train word vectors
path = '.\content_100_cut.txt'
sentences = word2vec.LineSentence(path)
model = Word2Vec(sentences, size=100,window=5,min_count=1)
model.save("word2vec.model")

Test word2vec model

In [6]:
with open('./content_100_cut.txt',encoding="utf-8") as f:
    content_100_cut = f.readlines()

In [7]:
model = Word2Vec.load(".\word2vec.model")

In [8]:
model.wv['新华社']

array([ 0.00329206,  0.00418664,  0.00080275,  0.00676559, -0.00234856,
        0.00114835,  0.0013668 ,  0.01309414,  0.0095053 ,  0.00530643,
       -0.00532372, -0.00337309, -0.00305863,  0.00089884,  0.00252425,
       -0.00199291,  0.00912812,  0.00595734,  0.00379461,  0.00785058,
       -0.00109401,  0.01002434,  0.00474972,  0.00063251, -0.00647363,
        0.00457432, -0.00889404,  0.01136271, -0.00535099,  0.00256104,
       -0.00090684, -0.00551009, -0.00094039,  0.0007965 ,  0.00366657,
        0.00602417,  0.00493478,  0.00159213,  0.00262246,  0.00511394,
        0.00017624, -0.01185923, -0.00192731,  0.00139888,  0.00595157,
        0.00062512, -0.00469247, -0.00070472, -0.00263998,  0.00357954,
        0.00011778,  0.00563376,  0.00699439, -0.01220829,  0.00062598,
       -0.00926866,  0.00621974, -0.0005826 ,  0.01394833, -0.0062076 ,
        0.01046599,  0.00396951,  0.00677602,  0.00126805, -0.00523894,
        0.00454702, -0.00024907, -0.00013272,  0.00587998, -0.00

In [16]:
model.similarity("互联网","公司")

d:\program files\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.03594041

In [33]:
for key in model.wv.similar_by_word('互联网',topn=10):
    print(key)

('黄伟明', 0.3289714455604553)
('数量', 0.32728031277656555)
('创意', 0.31198257207870483)
('一个', 0.3117627501487732)
('机器人', 0.3110935091972351)
('鼓励', 0.3107481300830841)
('政国', 0.3064036965370178)
('火箭', 0.3052319288253784)
('病毒', 0.3049807548522949)
('雅加达', 0.29981833696365356)


In [35]:
for key in model.wv.similar_by_word('机场',topn=10): 
    print(key)

('军事顾问', 0.3428409695625305)
('成功', 0.32130181789398193)
('浙江大学', 0.319980651140213)
('体积小', 0.2896482050418854)
('发出', 0.28459715843200684)
('孙奕', 0.283620148897171)
('训练', 0.2829975187778473)
('新金桥', 0.28276097774505615)
('超国家', 0.2807113528251648)
('宜昌', 0.27799803018569946)


测试结果词语相关性不是很大，可能是训练样本太少的原因？

<font color='red'>
词向量其实没必要从头自己训练，掌握了方法就好了，以后也很少自己训练，可以下载别人训练好的直接用，比如这个地址https://github.com/Embedding/Chinese-Word-Vectors；
</font>

**Step 2. TextRank**

$$WS(V_{i}) = (1-d) + d*\sum_{j \in In(V_{i})} \frac{w_{ji}}{\sum_{V_{k}\in Out(V_j)}w_{jk}}WS(V_{j})$$

对比PageRank：  
$$PR(P_i,t+1) = (1-d) + d*\sum_{j\in In(P_i)}\frac{1}{|Out(P_{j})|}PR(P_{j},t)$$

<font color='red'>
上一版问题:calPR 中少了一个归一化，要仔细对比一下公式
</font>

In [9]:
class TextRank(object):
    def __init__(self,word2vec,words,d,window,iternum):
        self.word2vec = word2vec
        self.words = words
        self.d = d
        self.window = window
        self.iternum = iternum
        self.edge_dict = {} #记录节点的边连接字典

    def createNodes(self):
        """
        create the nodes of a graphord2vec,words,d,window,iternum
        """
        tmp_list = []
        for i,word in enumerate(self.words):  #遍历每个词
            if word not in self.edge_dict.keys():
                tmp_list.append(word)
                tmp_set = set()
                left = i-(self.window-1)/2
                right = i + (self.window+1)/2
                if left<0:left=0
                if right>len(self.words):right=len(self.words)
                for j in range(int(left),int(right)):  #遍历每个窗口
                    if j==i:continue
                    tmp_set.add(self.words[j])
                self.edge_dict[word] = tmp_set


    def createMatrix(self):
        """
        create a Matrix representing the weights of edges based on the similarity
        """
        self.matrix = np.zeros([len(set(self.words)), len(set(self.words))])
        self.words_id = list(range(len(self.words)))
        self.word_index = dict(zip(self.words,self.words_id))  # 记录词的index
        self.index_dict = dict(zip(self.words_id,self.words)) # 记录节点index对应的词

        for key in self.edge_dict.keys():
            for w in self.edge_dict[key]:
                self.matrix[self.word_index[key]][self.word_index[w]] = self.word2vec.similarity(key,w)
                self.matrix[self.word_index[w]][self.word_index[key]] = self.word2vec.similarity(key,w)
                
        #归一化
        for j in range(self.matrix.shape[1]):
            sum = 0
            for i in range(self.matrix.shape[0]):
                sum += self.matrix[i][j]  #对每一列求和
            for i in range(self.matrix.shape[0]):
                self.matrix[i][j] /= sum  #对列进行归一化

    def calPR(self):
        self.PR = np.ones([len(self.words), 1])
        for i in range(self.iternum):
            self.PR = (1 - self.d) + self.d * np.dot(self.matrix, self.PR)

    def result(self):
        word_pr={}
        res = dict(zip(self.words,self.PR))
        print(res)
        return res
if __name__ == '__main__':
    model = Word2Vec.load('.\word2vec.model')
    words = [word for word, wv in model.wv.vocab.items()]  # 获取所有词
    tr = TextRank(model,words,d=0.85,window=5,iternum=1)
    tr.createNodes()
    tr.createMatrix()
    tr.calPR()
    tr.result()

d:\program files\python\python36\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
d:\program files\python\python36\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


{'新华社': array([0.69675488]), '联合国': array([0.67880589]), '５': array([1.49456956]), '月': array([1.16298712]), '１': array([1.34314167]), '日电': array([0.76702736]), '记者': array([1.37750014]), '史霄萌': array([0.22410765]), '中国': array([1.12699833]), '常驻': array([0.86117634]), '副': array([0.4984583]), '代表': array([3.17019934]), '吴': array([-0.87230053]), '海涛': array([2.31013833]), '日': array([0.13515646]), '中方': array([1.08812092]), '国际': array([1.64342734]), '社会': array([0.65982329]), '一道': array([-0.15789455]), '推进': array([1.92420572]), '反恐': array([1.36488762]), '合作': array([3.27766066]), '完善': array([0.78548421]), '防扩散': array([0.85586671]), '体系': array([0.9437062]), '维护': array([0.28115997]), '世界': array([-0.64265705]), '和平': array([1.68068056]), '稳定': array([-5.16724172]), 'n': array([-4.70740429]), '联合国安理会': array([1.84822465]), '当天': array([4.2351323]), '伊斯兰': array([8.24542445]), '国': array([-0.60188631]), '基地': array([0.5414291]), '组织': array([3.7432556]), '制裁': array([2.60982177])

#### 选做 1.  提取新闻人物里的对话。(使用以上提取小数据即可）

Demo：视频 2：15：23左右

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

#### 选择2. ： 电影评论分类。

在这个作业中你要完成一个电影评论分类任务。

1.数据获取。（采用爬虫技术爬取相关网页上的电影评论数据，例如猫眼电影评论，豆瓣电影评论）

2.把所获得数据分解为训练集，验证集和测试集。

3.选用相应算法构建模型，并测试。

#### 选择3：文章自动续写

在这个作业中你要完成一个文章自动续写的模型。

1.数据获取。（根据你的兴趣采用爬虫技术爬去相关网站上的文本数据内容：比如故事网站，小说网站等）

2.选取模型，并训练。

3.展示一些你模型的输出例子。